<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/Slicing_and_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import os
import pylab as pl
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm
import shutil
from collections import Counter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/MLTSA22/Final_Project

/content/drive/MyDrive/MLTSA22/Final_Project


In [29]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project

00000258_s002_t000.edf
all_tuh_eeg_seizure_links.txt
dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf
dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf
files_raw_channels_small_df.pkl
isip_tuh_eeg
seizures_types_v01.xlsx
temp


In [5]:
working_folder = Path('/content/drive/MyDrive/MLTSA22/Final_Project')

In [125]:
df = pd.read_pickle(
    working_folder.joinpath('files_raw_channels_small_df.pkl')
)

In [126]:
df.head()

,filename,category,configuration,index,patient_#,session,EEG FP1-REF,EEG FP2-REF,EEG F3-REF,EEG F4-REF,...,EEG T3-REF,EEG T4-REF,EEG T5-REF,EEG T6-REF,EEG FZ-REF,EEG CZ-REF,EEG PZ-REF,EEG A1-REF,EEG A2-REF,labels
file_name,,,,,,,,,,,,,,,,,,,,,
00010563_s002_t003,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[2.7726555877600295e-10, 1.3483732842699443e-0...","[-3.906051578001337e-10, 2.4807970640912983e-0...","[-1.9284389162122937e-10, 1.0694757868255472e-...","[-5.096480914428079e-11, 4.047438658809519e-06...",...,"[4.146017629996594e-09, -2.0413263864026096e-0...","[1.444056012902292e-09, 3.817044838007959e-07,...","[-2.910958452221951e-09, -3.250484807099009e-0...","[1.8423372397872572e-09, -7.1615811255855e-07,...","[-1.364304852525069e-10, 3.5260483039175664e-0...","[1.3531881762796245e-10, 6.983863806835304e-07...","[-2.3495119248263287e-10, -7.617383775562379e-...","[9.596059492877374e-10, -1.1888197912083615e-0...","[1.0519972902783143e-11, -2.1400490121479406e-...","[[0.0, 601.0, bckg]]"
00010563_s002_t004,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[-1.0278586830940462e-08, -1.7158036103551026e...","[2.7648617251111243e-09, -2.8490217427264524e-...","[3.288941204731503e-09, 4.617119704569293e-07,...","[3.399964489573073e-09, -5.345430061192608e-06...",...,"[8.658498750608017e-09, 1.7031133962228388e-07...","[1.780158122957924e-09, -2.152228442797806e-06...","[-4.916698865387427e-10, -2.9856703770832174e-...","[-6.258704822517387e-09, -1.8593219919890405e-...","[9.815260117792147e-10, -2.4137912796958633e-0...","[-2.357383015178091e-10, -5.355394184505534e-0...","[-1.0067458116184742e-09, -1.6493666048317114e...","[-4.7017904553916475e-09, -1.2061141044168896e...","[-1.1048988390260309e-08, -1.0048047870747276e...","[[0.0, 300.0, bckg]]"
00010563_s002_t005,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[3.7111311706420487e-09, -1.0474257222754713e-...","[7.219823200366963e-09, -3.1951117881894467e-0...","[2.1782993270312925e-09, -8.533492755460114e-0...","[-1.365985882998853e-09, 7.092433104769807e-06...",...,"[-8.948886072381209e-10, -7.939269490082801e-0...","[3.07454363042913e-09, 1.4140713109054442e-06,...","[6.674844103884688e-09, -4.037450817687943e-06...","[-3.61287734508141e-09, 6.247470115147276e-06,...","[-2.062432726832931e-09, 5.42573298081137e-07,...","[-8.346167589202069e-10, -3.2631357231385514e-...","[2.2304470728923623e-10, 3.7421245926520404e-0...","[1.4396886880297788e-09, -7.661145779436604e-0...","[6.327147966320168e-10, 1.1833053986962973e-05...","[[0.0, 601.0, bckg]]"
00010563_s003_t000,train__01_tcp_ar__105__00010563__s003_2013_07_...,train,01_tcp_ar,105,00010563,s003_2013_07_17,"[-4.837242318016071e-09, -1.9639659553544583e-...","[-1.7168631924658783e-09, 1.0420288689012175e-...","[-2.8357483900477646e-09, 1.3444340192373916e-...","[-4.3997702726288667e-10, -4.612748503245249e-...",...,"[-1.140736912419709e-09, 7.245232568610588e-07...","[2.6767669024911234e-09, 1.6102607124345142e-0...","[4.665380888551219e-09, 9.506710232528226e-07,...","[2.351093927809803e-09, 2.4800401575907515e-06...","[-1.0594241716063995e-10, -8.909057146788971e-...","[-4.790726036099662e-10, -1.4461463449301e-06,...","[4.02520870030525e-10, 1.1526165134603241e-08,...","[-5.771001920778108e-09, 3.624165205087055e-08...","[4.244714350965895e-09, -2.003986424867263e-06...","[[0.0, 300.0, bckg]]"
00010563_s003_t001,train__01_tcp_ar__105__00010563__s003_2013_07_...,train,01_tcp_ar,105,00010563,s003_2013_07_17,"[-5.683698910306182e-08, 2.654920991845486e-06...","[-3.337798055224077e-08, 2.8095137800880105e-0...","[3.7954605921399086e-08, 1.4489212326297262e-0...","[-1.2525189500901834e-08, 3.12269697219255e-06...",...,"[8.124202650339955e-08, 5.883211872988577e-07,...","[-4.9073445727403074e-08, 1.3883104676019777e-...","[1.9867273141848352e-08, 1.6805965603429214e-0...","[-6.6

In [127]:
df.columns

Index(['filename', 'category', 'configuration', 'index', 'patient_#',
       'session', 'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF', 'labels'],
      dtype='object')

In [128]:
# creating a new column, where there will be a list of labels
# the list will be the same for ALL the channels of a particular time series
df['seizure_type'] = np.NaN

In [129]:
df.columns

Index(['filename', 'category', 'configuration', 'index', 'patient_#',
       'session', 'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF', 'labels', 'seizure_type'],
      dtype='object')

Let's upload additional df with the types of the events/seizures.

In [31]:
seizure_types = pd.read_excel(
    working_folder.joinpath('seizures_types_v01.xlsx')
)

In [53]:
seizure_types

,Class Code,Class No.,Event Name,Signs,Locality,Description
0,NaN,0,No Event,NaN,NaN,An unclassified event
1,SPSW,1,Spike/Sharp and Wave,Electrographic,All,"Spike and wave/complexes , sharp and wave/comp..."
2,GPED,2,Generalized Periodic Epileptiform Discharges,Electrographic,Generalized,Diffused periodic discharges
3,PLED,3,Periodic Lateralized Epileptiform Discharges,Electrographic,Hemispheric/Focal,Focal periodic discharges
4,EYBL,4,Eye blink,Clinical & Electrographic,Focal,"A specific type of sharp, high amplitude eye m..."
5,ARTF,5,Artifacts (All),Clinical & Electrographic,All,"Any non-brain activity electrical signal, such..."
6,BCKG,6,Background,Electrographic,NaN,Baseline/non-interesting events
7,SEIZ,7,Seizure,Clinical &| Electrographic,All,Common seizure class which can include all typ...
8,FNSZ,8,Focal Non-Specific Seizure,Electrographic,Hemispheric/Focal,Focal seizures which cannot be specified with ...
9,GNSZ,9,Generalized Non-Specific Seizure,Electrographic,Generalized,Generalized seizures which cannot be further c...


In [52]:
seiz_names = seizure_types['Class Code'].values

In [77]:
# based on the classification in the table, these are epileptic seizures
names = ['SEIZ', 'FNSZ', 'GNSZ', 'SPSZ', 'CPSZ', 
              'ABSZ', 'TNSZ', 'CNSZ', 'TCSZ', 'ATSZ', 
              'MYSZ']

In [78]:
seiz_names = []
for name in names:
  seiz_names.append(name.lower())

In [79]:
seiz_names

['seiz',
 'fnsz',
 'gnsz',
 'spsz',
 'cpsz',
 'absz',
 'tnsz',
 'cnsz',
 'tcsz',
 'atsz',
 'mysz']

Column 'labels' have np.array files - matrices.

In [119]:
tryout = df.at['00010563_s002_t003', 'EEG P3-REF']

In [120]:
label = df.at['00010563_s002_t003', 'labels']

In [121]:
label.shape

(1, 3)

In [122]:
def labeling(label):
  sampl_freq = 200 # Fixed value after downsampling
  labels = [] # list for labels

  for row in range(0, label.shape[0]): # if there are several rows in label
    start = label[row][0] * sampl_freq # Hz = Number of samples per second
    end = label[row][1] * sampl_freq
    for i in seiz_names:
      if label[row][2] == i:    
        #conditions if there is a seizure type in the .tse file     
        if (start <= counter and
            end > counter and
            end <= counter+window_size):
            # 1 seizure starts before window and ends within
          labels.append(1)
        elif (start >= counter and
              end <= counter+window_size):
          # 2 seizure within window
          labels.append(1)
        elif (start >= counter and
            start < counter+window_size and
            end > counter+window_size):
          # 3 seizure starts before the end of window
          labels.append(1)
        elif (start <= counter and
              end >= counter+window_size):
          # 4 window is within seizure
          labels.append(1)
        else:
          # no seizure within the window
          labels.append(0)
      
        #if the type in the .tse file is not a seizure
      else: 
        labels.append(0)
      break

  return labels

In [123]:
def slicing(timeseries, label):
  # parameters for slicing 
  window_size = 100000
  step_size = 1000
  sliced = [] # list for timeseries chunks
  counter = 0

  labels = []

  while counter < len(tryout):
    series = tryout[counter:counter+window_size]
    std_series = (series - np.mean(series)) / np.std(series) # standardizing
    sliced.append(std_series)

    #labeling for slices
    labels.append(labeling(label))

    counter += window_size
    counter += step_size

  return sliced, labels

In [124]:
slicing(tryout, label)

([array([-0.00255815, -0.4606023 , -0.59290459, ...,  0.32461385,
          0.26107907,  0.36405002]),
  array([ 0.4280931 , -0.06268023, -0.05306376, ...,  0.17312215,
          0.12017823,  0.03218337])],
 [[0], [0]])